<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/face_landmark_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import tensorflow
from tensorflow import keras

In [3]:
y_train = np.load('/content/gdrive/MyDrive/data-v/y_train.npy')
frames = np.load('/content/gdrive/MyDrive/data-v/frames.npy')

In [4]:
img_size = 90

In [5]:
model_layers=[
    keras.layers.Conv2D( 256, input_shape=( img_size , img_size , 3 ) , kernel_size=( 5 , 5 ) , strides=1 , activation='relu',name="input_layer"),
    keras.layers.Conv2D( 256 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D( 256, kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256, kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D( 200, kernel_size=( 5 , 5 ) , strides=2 , activation='relu'),
    keras.layers.Conv2D( 200 , kernel_size=( 5 , 5 ) , strides= 1, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D( 200, kernel_size=( 5 , 5 ) , strides=1 , activation='relu'),
    keras.layers.Conv2D( 200 , kernel_size=( 5 , 5 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D( 170, kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 170, kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D( 136, kernel_size=( 3 , 3 ) , strides=1 , activation='relu'),
    keras.layers.Conv2D( 136, kernel_size=( 3 , 3 ) , strides=2 , activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D( 136, kernel_size=( 3 , 3 ) , strides=2 , activation='relu'),
    keras.layers.Conv2D( 136 , kernel_size=( 3 , 3 ) , strides=1 , activation='sigmoid'),

    
]
model=keras.Sequential(model_layers)
model.compile( loss= keras.losses.mean_squared_error , optimizer= keras.optimizers.Adam( lr=0.001 ) )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Conv2D)        (None, 86, 86, 256)       19456     
                                                                 
 conv2d (Conv2D)             (None, 82, 82, 256)       1638656   
                                                                 
 batch_normalization (BatchN  (None, 82, 82, 256)      1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 82, 82, 256)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 78, 78, 256)       1638656   
                                                                 
 conv2d_2 (Conv2D)           (None, 74, 74, 256)       1638656   
                                                        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
train=model.fit(frames,y_train,epochs=3,batch_size=32, verbose=2)

Epoch 1/3
69/69 - 70s - loss: 0.0040 - 70s/epoch - 1s/step
Epoch 2/3
69/69 - 70s - loss: 0.0034 - 70s/epoch - 1s/step
Epoch 3/3
69/69 - 70s - loss: 0.0033 - 70s/epoch - 1s/step
